In [2]:
import requests

def fetch_github_files(repo_url, directory):
    """
    Fetches files from a GitHub repository.

    Args:
        repo_url (str): The URL of the GitHub repository.
        directory (str): The directory containing the CADCAD model files.

    Returns:
        dict: A dictionary mapping file paths to their content.
    """
    base_url = 'https://api.github.com/repos/'
    files_content = {}

    url = f"{base_url}{repo_url}/contents/{directory}"
    headers = {'Accept': 'application/vnd.github.v3.raw'}

    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        # Parse the response to get the file paths
        files = response.json()

        for file_info in files:
            file_path = file_info['path']
            file_content_url = file_info['download_url']

            # Fetch file content
            file_content_response = requests.get(file_content_url)

            if file_content_response.status_code == 200:
                files_content[file_path] = file_content_response.text
            else:
                print(f"Failed to fetch content of {file_path}: {file_content_response.status_code}")
    else:
        print(f"Failed to fetch directory listing from {url}: {response.status_code}")

    return files_content

# GitHub repository URL
repo_url = 'BlockScience/neural-quorum-governance'

# Directory containing the CADCAD model files
directory = 'nqg_model/'

# Fetch files from the GitHub repository
files_content = fetch_github_files(repo_url, directory)
print(files_content)


{'nqg_model/__init__.py': 'from nqg_model.params import SINGLE_RUN_PARAMS, INITIAL_STATE, TIMESTEPS, SAMPLES\nfrom nqg_model.structure import NQG_MODEL_BLOCKS\n\ndefault_run_args = (INITIAL_STATE,\n                     {k: [v] for k, v in SINGLE_RUN_PARAMS.items()},\n                    NQG_MODEL_BLOCKS,\n                    TIMESTEPS,\n                    SAMPLES)', 'nqg_model/__main__.py': 'from nqg_model import default_run_args\nfrom nqg_model.experiment import standard_run\nfrom cadCAD_tools.execution import easy_run\nfrom datetime import datetime\nimport click\nimport os\n\n\n@click.command()\n@click.option(\'-e\', \'--experiment-run\', \'experiment_run\',\n              default=False,\n              is_flag=True,\n              help="Make an experiment run instead")\n@click.option(\'-p\', \'--pickle\', \'pickle\', default=False, is_flag=True)\ndef main(experiment_run: bool, pickle: bool) -> None:\n    timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")\n    if experiment_run

In [3]:
files_content

{'nqg_model/__init__.py': 'from nqg_model.params import SINGLE_RUN_PARAMS, INITIAL_STATE, TIMESTEPS, SAMPLES\nfrom nqg_model.structure import NQG_MODEL_BLOCKS\n\ndefault_run_args = (INITIAL_STATE,\n                     {k: [v] for k, v in SINGLE_RUN_PARAMS.items()},\n                    NQG_MODEL_BLOCKS,\n                    TIMESTEPS,\n                    SAMPLES)',
 'nqg_model/__main__.py': 'from nqg_model import default_run_args\nfrom nqg_model.experiment import standard_run\nfrom cadCAD_tools.execution import easy_run\nfrom datetime import datetime\nimport click\nimport os\n\n\n@click.command()\n@click.option(\'-e\', \'--experiment-run\', \'experiment_run\',\n              default=False,\n              is_flag=True,\n              help="Make an experiment run instead")\n@click.option(\'-p\', \'--pickle\', \'pickle\', default=False, is_flag=True)\ndef main(experiment_run: bool, pickle: bool) -> None:\n    timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")\n    if experiment_ru

In [1]:

import requests
import faiss
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import openai
# Define your OpenAI API key
openai.api_key = 'sk-proj-'

In [2]:

def fetch_github_files_recursive(repo_url, path=''):
    """
    Recursively fetch all files in a GitHub repository.

    Args:
        repo_url (str): The URL of the GitHub repository.
        path (str): The current directory path.

    Returns:
        dict: A dictionary mapping file paths to their content.
    """
    base_url = 'https://api.github.com/repos/'
    files_content = {}

    url = f"{base_url}{repo_url}/contents/{path}"
    headers = {'Accept': 'application/vnd.github.v3.raw'}

    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        items = response.json()

        for item in items:
            if item['type'] == 'file':
                file_path = item['path']
                file_content_url = item['download_url']

                file_content_response = requests.get(file_content_url)

                if file_content_response.status_code == 200:
                    files_content[file_path] = file_content_response.text
                else:
                    print(f"Failed to fetch content of {file_path}: {file_content_response.status_code}")
            elif item['type'] == 'dir':
                dir_path = item['path']
                sub_dir_files = fetch_github_files_recursive(repo_url, dir_path)
                files_content.update(sub_dir_files)
    else:
        print(f"Failed to fetch directory listing from {url}: {response.status_code}")

    return files_content

# Fetch files from the GitHub repository
repo_url = 'BlockScience/neural-quorum-governance'
files_content = fetch_github_files_recursive(repo_url)
# Save the files content locally if needed

In [3]:
files_content.keys()

dict_keys(['.gitignore', 'README.md', 'assets/collage_nqg_demo.png', 'assets/nqg.png', 'notebooks/proof-of-concept-demo.ipynb', 'notebooks/sanity_check.ipynb', 'nqg_model/__init__.py', 'nqg_model/__main__.py', 'nqg_model/experiment.py', 'nqg_model/helper.py', 'nqg_model/logic.py', 'nqg_model/neural_quorum_governance.py', 'nqg_model/params.py', 'nqg_model/structure.py', 'nqg_model/types.py', 'requirements.txt'])

In [5]:

import json
# import the OpenAI Python library for calling the OpenAI API
from openai import OpenAI
import os

client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY", "sk-proj-"))


In [6]:

def get_relevant_files(file_names):
    # Create the prompt
    prompt = f"""Here is a list of file names in a CADCAD project repository:
    {file_names}

    Please identify the files that are likely to contain:
    1. CADCAD Parameters
    2. CADCAD State Variables
    3. CADCAD State Update Blocks
    4. CADCAD Policies

    Return the file names categorized under each category as a dictionary.
    """

    # Send the prompt to OpenAI GPT-3.5 Turbo
    MODEL = "gpt-3.5-turbo"
    response = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": "You respond only with the categorized dictionary for python in json format."},
            {"role": "user", "content": prompt},
        ],
        temperature=0,
    )

    # Deserialize the JSON response
    response_dict = json.loads(response.model_dump_json())
    categorized_files=json.loads(response_dict['choices'][0]['message']['content'])


    return categorized_files


file_names = list(files_content.keys())

categorized_files = get_relevant_files(file_names)
print(categorized_files)


{'CADCAD Parameters': ['nqg_model/params.py'], 'CADCAD State Variables': ['nqg_model/types.py'], 'CADCAD State Update Blocks': ['nqg_model/logic.py'], 'CADCAD Policies': ['nqg_model/neural_quorum_governance.py']}


In [19]:
files_content['nqg_model/params.py']

'from nqg_model.types import *\nfrom nqg_model.neural_quorum_governance import DEFAULT_NG_LAYERS\nfrom numpy import nan\n\nTIMESTEPS = 100\nSAMPLES = 1\n\nN_INITIAL_USERS = 6\nN_PROJECTS = 15\nN_PAST_ROUNDS = 5\n\nAVERAGE_PAST_VOTES_PER_USER = 1.5\nPAST_ROUNDS = set(i for i in range(N_PAST_ROUNDS))\n\nDEFAULT_PROJECTS = set(f"proj_{i}" for i in range(N_PROJECTS))\n\n\nINITIAL_ORACLE_STATE = OracleState(\n    pagerank_results={},\n    reputation_bonus_map={\n    ReputationCategory.Tier3: 0.3,\n    ReputationCategory.Tier2: 0.2,\n    ReputationCategory.Tier1: 0.1,\n    ReputationCategory.Uncategorized: 0.0},\n    prior_voting_bonus_map={\n    1: 0.0,\n    2: 0.1,\n    3: 0.2,\n    4: 0.3\n},\nreputation_bonus_values={},\nprior_voting_bonus_values={}\n)\n\nINITIAL_STATE = NQGModelState(\n    days_passed=0.0,\n    delta_days=nan,\n    users=[],\n    user_round_decisions={},\n    delegatees={},\n    trustees={},\n    action_matrix={},\n    vote_matrix={},\n    per_project_voting={},\n    or

In [ ]:
# create a function create_hackmd_doc that takes a topic from the categorized_files dictionary and its key, then makes an openai call to generate a document that explains the topic

def create_hackmd_doc(topic, files_content):
    # Create the prompt
    prompt = f"""Please generate a document that explains the topic: {topic}. from the python file provided below:
    {files_content}
    """

    # Send the prompt to OpenAI GPT-3.5 Turbo
    MODEL = "gpt-3.5-turbo"
    response = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": "You respond only with the document for python in markdown format."},
            {"role": "user", "content": prompt},
        ],
        temperature=0,
    )

    # Deserialize the JSON response
    response_dict = json.loads(response.model_dump_json())
    doc = response_dict['choices'][0]['message']['content']

    return doc


docs = {}
for topic in categorized_files:
    print(f"Files related to {topic}:")
    for file_name in categorized_files[topic]:
        print(f"- {file_name}")
        # print(f"- - {files_content[file_name]}")
        doc = create_hackmd_doc(topic, files_content[file_name])
        docs[topic] = doc

In [23]:
docs

{'CADCAD Parameters': "```markdown\n# CADCAD Parameters\n\nIn the provided Python script, the CADCAD parameters for a Neural Quorum Governance (NQG) model are defined. These parameters are crucial for simulating the behavior of the governance system. Below are the key CADCAD parameters defined in the script:\n\n1. **TIMESTEPS**: The total number of timesteps for the simulation is set to 100.\n2. **SAMPLES**: The number of samples for the simulation is set to 1.\n\n### Initial User and Project Parameters\n3. **N_INITIAL_USERS**: Number of initial users in the system is set to 6.\n4. **N_PROJECTS**: Number of projects in the system is set to 15.\n5. **N_PAST_ROUNDS**: Number of past rounds considered in the simulation is set to 5.\n6. **AVERAGE_PAST_VOTES_PER_USER**: Average past votes per user is set to 1.5.\n\n### Default Project and Round Parameters\n7. **PAST_ROUNDS**: Set of past rounds based on the number of past rounds defined.\n8. **DEFAULT_PROJECTS**: Set of default projects nam

In [30]:
for topic, doc in docs.items():
    print(f"Document for {topic}:")
    # print(doc)
    # convert doc to markdown file
    with open(f"{topic}.md", "w") as f:
        f.write(doc)

Document for CADCAD Parameters:
Document for CADCAD State Variables:
Document for CADCAD State Update Blocks:
Document for CADCAD Policies:
